# **環境設定**

In [4]:
#安裝套件與引用yolo
!pip install ultralytics

from IPython import display
display.clear_output()
from IPython.display import display, Image

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.1.19 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.5/107.7 GB disk)


In [6]:
#建立資料集路徑，記得替換{HOME}
HOME= '/content'
!mkdir {HOME}/datasets
%cd {HOME}/datasets

#下載roboflow套件並引入自定義圖片(查看說明文件第一點)
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="your api key")
project = rf.workspace("project-tx2ls").project("test_1112_01_apical-lesion_multi")
dataset = project.version(1).download("yolov8-obb")

/content/datasets
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to test_1112_01_apical-lesion_multi-1 in yolov8-obb:: 100%|██████████| 280/280 [00:00<00:00, 8570.86it/s]


# **訓練自定義模型**
查看說明文件第二點更改設定(data.yaml)

In [ ]:
# mode=train
%cd {HOME}

!yolo task=obb mode=train model=yolov8s-obb.pt data={dataset.location}/data.yaml epochs=10 imgsz=640 batch=8

/content
100% 22.2M/22.2M [00:00<00:00, 46.4MB/s]
Ultralytics YOLOv8.1.19 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=obb, mode=train, model=yolov8s-obb.pt, data=/content/datasets/test_1112_01_apical-lesion_multi-1/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

In [ ]:
# 查看訓練結果，記得注意路徑
%cd {HOME}
Image(filename=f'{HOME}/runs/obb/train2/confusion_matrix.png', width=600)

In [ ]:
# mode=val，記得注意路徑
%cd {HOME}

!yolo task=obb mode=val model={HOME}/runs/obb/train2/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
# mode=predict，記得注意路徑
%cd {HOME}
!yolo task=obb mode=predict model={HOME}/runs/obb/train2/weights/best.pt conf=0.5 source={dataset.location}/test/images save=true

In [ ]:
# 將測試結果一張張輸出給我們看
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/obb/predict/*.jpg'):
      display(Image(filename=image_path))
      print("\n")

# **額外功能**

In [ ]:
#這格選用 他是為了把之前的yolov5 OBB 轉 yolov8 OBB，現在roboflow已經有相關的匯出支援(若我們更改使用工具可能會用到)
import os

# Define the classes and their corresponding indices
class_names = ['0: apical_lesion']
class_indices = {name: index for index, name in enumerate(class_names)}


# Image dimensions
img_width, img_height = 640, 640

# Path to the folder containing the annotation files
folder_path = '/content/datasets/test_1112_01_apical-lesion_multi-1/valid/labels'

# Function to normalize coordinates
def normalize_coordinates(coord, max_value):
    return float(coord) / max_value

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # assuming the annotation files are .txt files
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()


        new_lines = []
        for line in lines:
            parts = line.strip().split(' ')
            if len(parts) == 10:
                # Extract label and coordinates
                label = parts[-2]
                coords = parts[:8]

                # Get the class index
                class_index = class_indices.get(label, -1)
                if class_index != -1:
                    # Normalize coordinates
                    normalized_coords = [normalize_coordinates(coords[i], img_width if i % 2 == 0 else img_height) for i in range(8)]

                    # Convert to the desired format and add to new lines
                    new_line = f"{class_index} " + " ".join(map(str, normalized_coords))
                    new_lines.append(new_line)

        # Write the converted lines to a new file or overwrite the existing file
        with open(file_path, 'w') as file:
            file.write('\n'.join(new_lines))